---
# Random problem generator

In [ ]:
using Random
using LinearAlgebra
using Distributions

"""
@article{Lewandowski2009a,
    Author = {Daniel Lewandowski and Dorota Kurowicka and Harry Joe},
    Journal = {Journal of Multivariate Analysis},
    Number = {9},
    Pages = {1989 - 2001},
    Title = {Generating random correlation matrices based on vines and extended onion method},
    Volume = {100},
    Year = {2009}}
https://doi.org/10.1016/j.jmva.2009.04.008
"""
function onion(n; η = 1.0)
    S = Symmetric(ones(n, n))
    β = η + (n-2)/2
    S.data[1,2] = 2*rand(Beta(β,β)) - 1
    for k = 2:n-1
        β -= 0.5
        y = rand(Beta(k/2, β))
        u = normalize!(randn(k))
        w = sqrt(y)*u
        F = cholesky(Symmetric(S[1:k,1:k]))
        S.data[1:k,k+1] = F.L*w
    end
    return S
end

function randncm(n; method=:onion, seed=0, γ=0.0, p=0.5)
    rng = Random.seed!(seed)
        
    # Random target matrix
    if method == :onion
        U = onion(n)
    else
        U = 2*rand(n,n) .- 1; U = Symmetric(triu(U,1) + I)
    end

    # Random noise
    E = Symmetric(triu(randn(n,n),1))
    U.data .= (1-γ).*U .+ γ.*E
    U = Symmetric(triu(U,1) + I)
    
    # Random sparse H
    H = [rand()<p ? 1.0 : 0.0 for i=1:n, j=1:n]
    H = Symmetric(triu(H,1))
    
    Random.seed!(rng)
    
    return U, H
end

In [ ]:
n, γ = 6, 0.0
U, H = randncm(n, γ=γ)
display(U)
display(H);

---
# JuMP.jl

In [ ]:
using JuMP, CSDP

function ncm(U, H; verbose=false)
    # Initialize model with correlation matrix constraints
    m = Model(with_optimizer(CSDP.Optimizer))
    if !verbose
        set_silent(m)
    end
    @variable(m, X[1:n,1:n], Symmetric)
    @constraint(m, diagcon, diag(X) .== 1)
    @constraint(m, psdcon, X in PSDCone())
    R = H.*(X .- U)
    @objective(m, Min, 0.5*dot(R, R))
    optimize!(m)
    Xval = value.(X)
    y = dual.(diagcon)
    Λ = dual.(psdcon)
    return Xval, y, Λ
end

In [ ]:
n, γ = 6, 0.5
U, H = randncm(n, γ=γ)
display(H.*U)
X, y, Λ = ncm(U, H);
display(H.*X);

In [ ]:
function optimalityconditions(U, H, X, y, Λ)
    rp = norm(1 .- diag(X))
    rd = norm(H.*H.*(X .- U) .+ Diagonal(y) .- Λ)
    @show rp
    @show rd
    @show dot(X, Λ)
    @show minimum(eigvals(X))
    @show minimum(eigvals(Λ))
    return nothing
end

In [ ]:
optimalityconditions(U, H, X, -y, Λ)

---

# ProjPSD

In [ ]:
struct ProjPSD
    nmax::Int
    jobz::Char
    range::Char
    il::Int
    iu::Int
    abstol::Float64
    m::Base.RefValue{Int}
    w::Vector{Float64}
    Z::Matrix{Float64}
    ldz::Int
    isuppz::Vector{Int}
    work::Vector{Float64}
    lwork::Int
    iwork::Vector{Int}
    liwork::Int
    info::Base.RefValue{Int}

    function ProjPSD(nmax)
        n = nmax
        A = Symmetric(zeros(1,1))

        jobz = 'V'
        range = 'V'
        lda = n
        vl = 0.0
        vu = Inf
        il = 0
        iu = 0
        abstol = -1.0
        m = Ref{Int}(0)
        w = zeros(n)
        Z = zeros(n, n)
        ldz = n
        isuppz = zeros(Int, 2n)
        work   = zeros(1)
        iwork  = zeros(Int, 1)
        info   = Ref{Int}(0)

        # Perform an optimal workspace query
        ccall((:dsyevr_64_, "libopenblas64_"), Cvoid,
            (Ref{UInt8}, Ref{UInt8}, Ref{UInt8}, Ref{Int},
                Ptr{Float64}, Ref{Int}, Ref{Float64}, Ref{Float64},
                Ref{Int}, Ref{Int}, Ref{Float64}, Ptr{Int},
                Ptr{Float64}, Ptr{Float64}, Ref{Int}, Ptr{Int},
                Ptr{Float64}, Ref{Int}, Ptr{Int}, Ref{Int},
                Ptr{Int}),
            jobz, range, A.uplo, n,
            A.data, lda, vl, vu,
            il, iu, abstol, m,
            w, Z, ldz, isuppz,
            work, -1, iwork, -1,
            info)

        lwork = Int(real(work[1]))
        liwork = iwork[1]

        resize!(work, lwork)
        resize!(iwork, liwork)

        new(nmax, jobz, range, il, iu, abstol, m, w, Z, ldz,
            isuppz, work, lwork, iwork, liwork, info)
    end
end


function (obj::ProjPSD)(A::Symmetric)
    lda, n = size(A)

    @assert lda == n
    @assert n <= obj.nmax

    vl = 0.0
    vu = Inf  # Tests show that vu = Inf is faster than
              # vu = min(norminf, normfro)
    abstol = -1.0

    #=
    vl = 1e-8
    norminf = ccall((:dlansy_64_, "libopenblas64_"), Cdouble,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, Ptr{Float64}, Ref{Int},
        Ptr{Float64}), 'I', A.uplo, n, A.data, lda, obj.work)
    normfro = ccall((:dlansy_64_, "libopenblas64_"), Cdouble,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, Ptr{Float64}, Ref{Int},
        Ptr{Float64}), 'F', A.uplo, n, A.data, lda, obj.work)
    vu = min(norminf, normfro)
    if vu < vl
        vu = 2*vl
    end
    abstol = 1e-8
    =#

    ccall((:dsyevr_64_, "libopenblas64_"), Cvoid,
    (Ref{UInt8}, Ref{UInt8}, Ref{UInt8}, Ref{Int},
        Ptr{Float64}, Ref{Int}, Ref{Float64}, Ref{Float64},
        Ref{Int}, Ref{Int}, Ref{Float64}, Ptr{Int},
        Ptr{Float64}, Ptr{Float64}, Ref{Int}, Ptr{Int},
        Ptr{Float64}, Ref{Int}, Ptr{Int}, Ref{Int},
        Ptr{Int}),
    obj.jobz, obj.range, A.uplo, n,
    A.data, lda, vl, vu,
    obj.il, obj.iu, abstol, obj.m,
    obj.w, obj.Z, obj.ldz, obj.isuppz,
    obj.work, obj.lwork, obj.iwork, obj.liwork,
    obj.info)

    k = obj.m[]
    λ, V = obj.w, obj.Z
    ldv = obj.ldz

    # V = V*diagm(sqrt.(λ))
    for j = 1:k
        tmp = sqrt(λ[j])
        for i = 1:n
            V[i,j] *= tmp
        end
    end

    # A = V*V'
    ccall((:dsyrk_64_, "libopenblas64_"), Cvoid,
        (Ref{UInt8}, Ref{UInt8}, Ref{Int}, Ref{Int}, Ref{Float64},
            Ptr{Float64}, Ref{Int}, Ref{Float64}, Ptr{Float64}, Ref{Int}),
        A.uplo, 'N', n, k, 1.0, V, ldv, 0.0, A.data, lda)

    return A
end

n = 10

A = Symmetric(randn(n,n))
B = copy(A)

@time myproj = ProjPSD(n);
@time myproj(B) # Compile myproj
@time myproj(A);

---
# Helper functions

In [ ]:
function plusdiag!(M::AbstractArray{T,2}, y::Vector{T}) where T
    for i=eachindex(y)
        @inbounds M[i,i] += y[i]
    end
    return M
end

# Replaces linesearch function evaluations with last function evaluation for cleaner plots
function cleanvals!(vals, linesearchcalls)
    fgcalls = sum(linesearchcalls)
    v = view(vals, length(vals)-fgcalls+1:length(vals))
    a = 1
    for numcalls in linesearchcalls
        numcalls==0 && break
        b = a + numcalls - 1
        fill!(view(v, a:b), v[b])
        a = b + 1
    end
    return vals
end

---
# LBFGSB.jl

In [ ]:
using Printf, LBFGSB

const START = b"START"
const FG    = b"FG"
const STOP  = b"STOP"
const NEW_X = b"NEW_X"

const STOP2  = b"ST"
const NEW_X2 = b"NE"

function copyval!(x, copytoinds, copyfromind)
    for i = copytoinds
        @inbounds x[i] = x[copyfromind]
    end
end

function calllbfgsb!(func!, g, y, 
        H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V, 
        fgcount, fvals, resvals, rpvals, rdvals, L, τ, α, σ,
        n, memlim, wa, iwa, nbd, lower, upper, task, task2, csave, lsave, isave, dsave,
        nRef, mRef, iprint, fRef, factr, pgtol;
        method=:IAPG,
        maxfgcalls=100,
        gtol=1e-2,
        exact=false,
        verbose=false,
        cleanvals=true,
    )

    fgcalls = 0
    linesearchcount = 0

    StopBFGS = false
    successful = true

    # "We start the iteration by initializing task."
    copyto!(task, START)

    while !StopBFGS

        # This is the call to the L-BFGS-B code.
        setulb(nRef, mRef, y, lower, upper, nbd,
            fRef, g, factr, pgtol, wa, iwa, task,
            iprint, csave, lsave, isave, dsave)

        if cleanvals && linesearchcount > 1
            a, b = fgcount[1]-linesearchcount+1, fgcount[1]
            copytoinds = a:b-1
            copyval!(fvals,   copytoinds, b)
            copyval!(resvals, copytoinds, b)
            copyval!(rpvals,  copytoinds, b)
            copyval!(rdvals,  copytoinds, b)
        end

        if task2 == FG
            if fgcalls >= maxfgcalls
                copyto!(task, STOP)
            else
                fRef[] = func!(g, y, 
                    H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V, 
                    fgcount, fvals, resvals, rpvals, rdvals, L, τ)

                fgcalls += 1
                if fgcalls > 1
                    linesearchcount += 1
                end

                if verbose
                    if fgcalls == 1
                        @printf("\n%8s %10s %10s %10s", 
                            "fgcalls", "fRef[]", "norm(g)", "gtol")
                    end
                    @printf("\n%8d %10.2e %10.2e %10.2e", 
                        fgcalls, fRef[], norm(g), gtol)
                end

                if !exact                
                    if method==:IAPG
                        condition = (norm(g) < gtol)
                    else
                        ε = max(0.0, dot(Xnew, Λ))
                        δ = norm(V)
                        dist = norm(M.data .= Xnew .- Y)
                        if method==:IR
                            condition = ((τ*δ)^2 + 2τ*ε*L ≤ L*((1-τ)*L - α*τ)*dist^2)
                            verbose && @printf(" %10.2e, %10.2e",
                                (τ*δ)^2 + 2τ*ε*L, L*((1-τ)*L - α*τ)*dist^2)
                        else # method==:IER
                            M.data .+= α.*V
                            β = norm(M)
                            condition = (β^2 + 2α*ε ≤ (σ*dist)^2)
                            verbose && @printf(" %10.2e, %10.2e",
                                β^2 + 2α*ε, (σ*dist)^2)
                        end
                    end

                    if condition
                        copyto!(task, STOP)
                    end
                end
            end

        elseif task2 == NEW_X2
            verbose && @printf(" (linesearch complete)")
            linesearchcount = 0

        else
            StopBFGS = true
            if !exact && task2 != STOP2
                verbose && @printf("\ntask = %s\n", String(copy(task)))
                successful = false
            end
        end

    end
    verbose && @printf("\n")

    return successful
end

---
# Nearest Correlation Matrix

In [ ]:
struct NCMstorage
    n::Int
    memlim::Int
    y::Vector{Float64}
    g::Vector{Float64}
    d::Vector{Float64}
    M::Symmetric{Float64,Array{Float64,2}}
    H2::Symmetric{Float64,Array{Float64,2}}
    Y::Symmetric{Float64,Array{Float64,2}}
    ∇fY::Symmetric{Float64,Array{Float64,2}}
    Λ::Symmetric{Float64,Array{Float64,2}}
    Γ::Symmetric{Float64,Array{Float64,2}}
    V::Symmetric{Float64,Array{Float64,2}}
    X::Symmetric{Float64,Array{Float64,2}}
    Xold::Symmetric{Float64,Array{Float64,2}}
    Xnew::Symmetric{Float64,Array{Float64,2}}
    wa::Vector{Float64}
    iwa::Vector{Int32}
    nbd::Vector{Int32}
    lower::Vector{Float64}
    upper::Vector{Float64}
    task::Vector{UInt8}
    task2::SubArray{UInt8,1,Vector{UInt8},Tuple{UnitRange{Int64}},true}
    csave::Vector{UInt8}
    lsave::Vector{Int32}
    isave::Vector{Int32}
    dsave::Vector{Float64}

    function NCMstorage(n, memlim)
        y = zeros(n)
        g = zeros(n)
        d = zeros(n)

        M = Symmetric(zeros(n,n))
        H2 = copy(M)
        Y = copy(M)
        ∇fY = copy(M)
        Λ = copy(M)
        Γ = copy(M)
        V = copy(M)
        X = copy(M)
        Xold = copy(M)
        Xnew = copy(M)

        nmax = n
        mmax = memlim

        wa = zeros(Cdouble, 2mmax*nmax + 5nmax + 11mmax*mmax + 8mmax)
        iwa = zeros(Cint, 3nmax)

        # provide nbd which defines the bounds on the variables:
        nbd = zeros(Cint, nmax)         # no bounds on the variables
        lower = zeros(Cdouble, nmax)    # the lower bounds
        upper = zeros(Cdouble, nmax)    # the upper bounds

        task  = fill(Cuchar(' '), 60)   # fortran's blank padding
        task2 = view(task, 1:2)
        csave = fill(Cuchar(' '), 60)   # fortran's blank padding
        lsave = zeros(Cint, 4)
        isave = zeros(Cint, 44)
        dsave = zeros(Cdouble, 29)

        new(n, memlim, y, g, d, M, H2, Y, ∇fY, Λ, Γ, V, X, Xold, Xnew, 
            wa, iwa, nbd, lower, upper, task, task2, csave, lsave, isave, dsave)
    end
end


struct NCMresults
    X::Symmetric{Float64,Array{Float64,2}}
    y::Vector{Float64}
    Λ::Symmetric{Float64,Array{Float64,2}}
    fvals::Vector{Float64}
    resvals::Vector{Float64}
end


function ncm(U::AbstractArray{T,2}, H::AbstractArray{T,2}, myproj::ProjPSD, storage::NCMstorage; 
        method=:IAPG, 
        exact=false,
        τ=1.0,
        α=0.0,
        σ=1.0,
        tol=1e-2,
        kmax=4000, 
        f_calls_limit=8000, 
        verbose=false,
        lbfgsbverbose=false,
        iprint=-1,
        cleanvals=true,
    ) where T

    # Loss function and gradient
    #f(X) = 0.5*norm(H.*(X .- U))^2
    #∇f(X) = Symmetric(H2.*(X .- U))

    # Check for valid input
    n = size(U, 1)
    n==storage.n || error("n != storage.n")
    size(U)==size(H) || error("U and H must be the same size")
    issymmetric(U) || error("U must be symmetric")
    issymmetric(H) || error("H must be symmetric")
    any(hij != 0.0 for hij in H) || error("H must be nonzero")

    method in [:IAPG, :IR, :IER] || error("method must be :IAPG, :IR, or :IER")
    verbose && println("$method method")

    y = storage.y
    g = storage.g
    d = storage.d
    M = storage.M
    H2 = storage.H2
    Y = storage.Y
    ∇fY = storage.∇fY
    Λ = storage.Λ
    Γ = storage.Γ
    V = storage.V
    X = storage.X
    Xold = storage.Xold
    Xnew = storage.Xnew

    fill!(y, 0)
    fill!(g, 0)
    fill!(d, 0)
    fill!(M, 0)
    fill!(H2, 0)
    fill!(Y, 0)
    fill!(∇fY, 0)
    fill!(Λ, 0)
    fill!(Γ, 0)
    fill!(V, 0)
    fill!(X, 0)
    fill!(Xold, 0)
    fill!(Xnew, 0)

    memlim = storage.memlim
    wa = storage.wa
    iwa = storage.iwa
    nbd = storage.nbd
    lower = storage.lower
    upper = storage.upper
    task = storage.task
    task2 = storage.task2
    csave = storage.csave
    lsave = storage.lsave
    isave = storage.isave
    dsave = storage.dsave

    nRef = Ref{Cint}(n)
    mRef = Ref{Cint}(memlim)
    iprint = Ref{Cint}(iprint)
    fRef = Ref{Cdouble}(0.0)
    factr = Ref{Cdouble}(0.0)
    pgtol = Ref{Cdouble}(0.0)

    H2.data .= H.^2

    # Lipschitz constant of ∇f
    L = norm(H2)

    if method==:IAPG
        τ==1 || error("IAPG method requires τ = 1")
        t0 = 1.0
    end

    if method==:IR
        0 < τ ≤ 1 || error("IR method requires 0 < τ ≤ 1")
        0 ≤ α ≤ (1 - τ)*(L/τ) || error("IR method requires 0 ≤ α ≤ $((1 - τ)*(L/τ))")
        t0 = 1.0
    end

    if method==:IER
        τ==1 || error("IER method requires τ = 1")
        α > 1/L || error("IER method requires α > $(1/L)")
        0 ≤ σ ≤ 1 || error("IER method requires 0 ≤ σ ≤ 1")
        λ = α/(1 + α*L)
        t0 = 0.0
    end

    # Evaluates dual objective function and its gradient
    function dualobj!(gg, y, 
            H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V, 
            fgcount, fvals, resvals, rpvals, rdvals, L, τ)

        fgcount[1] += 1

        τdL = τ/L
        Ldτ = L/τ

        #=
        ∇fY.data .= H2.*(Y .- U)
        M .= ∇fY; plusdiag!(M, y)  # M = ∇f(Y) + Diag(y)
        M.data .= Y .- τdL.*M      # M = Y - (τ/L)*(∇f(Y) + Diag(y))
        X .= M
        =#
        @inbounds for j=1:n
            for i=1:j
                ∇fY.data[i,j] = H2[i,j]*(Y[i,j] - U[i,j])
                M.data[i,j] = Y[i,j] - τdL*∇fY[i,j]
                X.data[i,j] = M[i,j]
            end
            M.data[j,j] -= τdL*y[j]
            X.data[j,j] = M[j,j]
        end
        myproj(X)

        # Update Λ and Γ
        #Λ.data .= Ldτ.*(X .- M)         # Λ is psd
        #Γ.data .= .-Λ; plusdiag!(Γ, y)  # Γ = Diag(y) - Λ

        # Ensure that diag(Xnew).==1 exactly
        @inbounds for j=1:n
            if X[j,j] > 0.0
                d[j] = 1.0/sqrt(X[j,j])
            else
                d[j] = 1.0
            end
        end
        @inbounds for j=1:n
            for i=1:j
                Λ.data[i,j] = Ldτ*(X[i,j] - M[i,j])
                Γ.data[i,j] = -Λ[i,j]
                Xnew.data[i,j] = d[i]*d[j]*X[i,j]
                V.data[i,j] = ∇fY[i,j] + Ldτ*(Xnew[i,j] - Y[i,j]) + Γ[i,j]
                M.data[i,j] = H[i,j]*(Xnew[i,j] - U[i,j])
            end
            Γ.data[j,j] += y[j]
            V.data[j,j] += y[j]
        end

        # Update V
        # V.data .= ∇fY .+ Ldτ.*(Xnew .- Y) .+ Γ

        # Compute and store the objective function
        #M.data .= H.*(Xnew .- U)
        fvals[fgcount[1]] = 0.5*dot(M,M)

        # Compute and store the optim. cond. residual
        @inbounds for j=1:n
            d[j] = 1.0 - Xnew[j,j]
            for i=1:j
                M.data[i,j] = H[i,j]*M[i,j] + Γ[i,j]
            end
        end
        rpvals[fgcount[1]] = norm(d)/(1 + √n)
        #M.data .= H2.*(Xnew .- U) .+ Γ
        rdvals[fgcount[1]] = norm(M)
        resvals[fgcount[1]] = max(rpvals[fgcount[1]],rdvals[fgcount[1]])

        # Compute the gradient of the dual function
        @inbounds for j=1:n
            gg[j] = 1.0 - X[j,j]
        end

        w, inds = myproj.w, 1:myproj.m[]
        return sum(y) + 0.5*Ldτ*dot(w,inds,w,inds)
    end

    k = 0
    t = t0
    gtol = NaN
    rp = rd = Inf
    innersuccess = true

    fgcount = [0]
    fvals   = Vector{Float64}(undef, f_calls_limit)
    resvals = Vector{Float64}(undef, f_calls_limit)
    rpvals  = Vector{Float64}(undef, f_calls_limit)
    rdvals  = Vector{Float64}(undef, f_calls_limit)

    while ( #innersuccess && 
            max(rp, rd) > tol && 
            k < kmax && 
            fgcount[1] < f_calls_limit )

        k += 1

        if method==:IER
            tnew = t + (λ + √(λ^2 + 4λ*t))/2
            Y.data .= (t/tnew).*Xnew .+ ((tnew - t)/tnew).*Xold
        else
            tnew = (1 + √(1 + 4t^2))/2
        end

        # Reset L-BFGS-B arrays
        fill!(wa, 0.0)
        fill!(iwa, 0)
        fill!(task, Cuchar(' '))
        fill!(csave, Cuchar(' '))
        fill!(lsave, 0)
        fill!(isave, 0)
        fill!(dsave, 0.0)

        maxfgcalls = f_calls_limit - fgcount[1]

        if method==:IAPG
            gtol = (1 + √n)*min(1/tnew^3.1, 0.2*rd)
        end

        if exact
            gtol = 0.0
        end

        # Solve the subproblem
        innersuccess = calllbfgsb!(dualobj!, g, y,
            H2, Y, U, ∇fY, M, X, Λ, Γ, d, Xnew, V,
            fgcount, fvals, resvals, rpvals, rdvals, L, τ, α, σ,
            n, memlim, wa, iwa, nbd, lower, upper, task, task2, csave, lsave, isave, dsave,
            nRef, mRef, iprint, fRef, factr, pgtol;
            method=method,
            maxfgcalls=maxfgcalls,
            gtol=gtol,
            exact=exact,            
            verbose=lbfgsbverbose,
            cleanvals=cleanvals,
        )
        if !innersuccess
            verbose && println("Failed to solve subproblem.")
        end
        fgcalls = fgcount[1] - (f_calls_limit - maxfgcalls)

        rp = rpvals[fgcount[1]]
        rd = rdvals[fgcount[1]]
        ε = dot(Xnew, Λ)
        δ = norm(V)
        dist = norm(M.data .= Xnew .- Y)

        if verbose
            mod(k, 20)==1 &&
            @printf("%4s %8s %10s %10s %14s %10s %10s %10s %10s %10s\n", 
                "k", "fgcalls", "||g||", "gtol", "f(X)", "rp", "rd", "<X,Λ>", "||V||", "||X-Y||")
            @printf("%4d %8d %10.2e %10.2e %14.6e %10.2e %10.2e %10.2e %10.2e %10.2e\n", 
                k, fgcalls, norm(g), gtol, fvals[fgcount[1]], rp, rd, ε, δ, dist)
        end

        if method==:IR
            ε = max(0.0, ε)
            condition = ((τ*δ)^2 + 2τ*ε*L ≤ L*((1-τ)*L - α*τ)*dist^2)
            if !condition && (fgcount[1] < f_calls_limit)
                verbose && println("WARNING: (τ*δ)^2 + 2τ*ε*L ≤ L*((1-τ)*L - α*τ)*dist^2 fails")
            end
        end

        if method==:IER
            ε = max(0.0, ε)
            M.data .+= α.*V
            β = norm(M)
            condition = (β^2 + 2α*ε ≤ (σ*dist)^2)
            if !condition && (fgcount[1] < f_calls_limit)
                verbose && println("WARNING: β^2 + 2α*ε ≤ (σ*dist)^2 fails")
            end
        end

        # Update
        if method==:IAPG
            Y.data .= Xnew .+ ((t - 1)/tnew).*(Xnew .- Xold)
            Xold .= Xnew
        end

        if method==:IR
            Y.data .= Xnew .- ((t/tnew)*(τ/L)).*V .+ ((t - 1)/tnew).*(Xnew .- Xold)
            Xold .= Xnew
        end

        if method==:IER
            Xold.data .-= (tnew - t).*(V .+ L.*(Y .- Xnew))
        end

        t = tnew
    end

    if max(rp, rd) > tol
        verbose && println("Failed to converge after $(fgcount[1]) function evaluations.")
    else
        verbose && println("Converged after $(fgcount[1]) function evaluations.")
    end

    resize!(fvals, fgcount[1])
    resize!(resvals, fgcount[1])

    return NCMresults(Xnew, y, Λ, fvals, resvals)
end

---
# Testing

In [ ]:
n, γ = 100, 0.1
memlim = 10

U, H = randncm(n, γ=γ)

myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

@time res = ncm(U, H, myproj, storage)
@show length(res.fvals)
optimalityconditions(U, H, res.X, res.y, res.Λ)
println()
@time res = ncm(U, H, myproj, storage, method=:IR, τ=0.95)
@show length(res.fvals)
optimalityconditions(U, H, res.X, res.y, res.Λ)

In [ ]:
using Plots, LaTeXStrings

n, γ = 100, 0.1
memlim = 10

U, H = randncm(n, γ=γ)

myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

plot(yaxis=:log, size=(900,600), title="n=$n, \\gamma=$γ", 
    xlabel="function evaluations", ylabel=L"\max\{R_p,R_d\}")
@time res = ncm(U, H, myproj, storage);
plot!(1:length(res.resvals), res.resvals, label="IAPG")
@time res = ncm(U, H, myproj, storage, method=:IR, τ=0.95)
plot!(1:length(res.resvals), res.resvals, label="IR, \\tau=0.95")

In [ ]:
n, γ = 200, 0.1
memlim = 10
U, H = randncm(n, γ=γ, seed=0)
myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

f_calls_limit=kmax=8000
verbose=false
tol=1e-2

plt = plot(yaxis=:log, legend=:topright, size=(900,600), title="n=$n, \\gamma=$γ",
    xlabel="function evaluations", ylabel=L"\max\{R_p,R_d\}")

@time res = ncm(U, H, myproj, storage, 
    method=:IAPG, 
    f_calls_limit=f_calls_limit, 
    kmax=kmax,
    tol=tol,
    verbose=verbose,
)
plot!(1:length(res.resvals), res.resvals, label="IAPG")

for τ in [0.95]
    @time res = ncm(U, H, myproj, storage, 
        method=:IR,
        τ=τ, 
        f_calls_limit=f_calls_limit,
        kmax=kmax,
        tol=tol,
        verbose=verbose,
    )
    plot!(1:length(res.resvals), res.resvals, label="IR, \\tau=$τ")
end
plt

In [ ]:
n, γ = 200, 0.1
a, b, N = 1e-2, 1e-0, 9

memlim = 10

myproj = ProjPSD(n)
storage = NCMstorage(n, memlim)

algos = [(:IAPG, 1.0), (:IR, 0.95)]
tols = exp10.(range(log10(a), stop=log10(b), length=N))
seeds = 1:1

M = zeros(Int, length(tols), length(algos), length(seeds))

algonames = string.([algo[1] for algo in algos])
@printf("%4s %8s %8s %8s\n", "", "tol", algonames...)
for (i, tol) in enumerate(tols)
    @printf("%4d %8.4f", i, tol)
    for (j, algo) in enumerate(algos)
        method, τ = algo
        for (k, seed) in enumerate(seeds)
            U, H = randncm(n, γ=γ, seed=seed)
            res = ncm(U, H, myproj, storage, method=method, τ=τ, tol=tol)
            if res.resvals[end] > tol
                @show tol, algo, seed
            end
            M[i,j,k] = length(res.fvals)
        end
        @printf(" %8.1f", mean(M[i,j,:]))
    end
    @printf("\n")
end

In [ ]:
xticks = [1e-2, 1e-1, 1e-0]

num2str(x) = @sprintf("%.0e", x)

plt = plot(xaxis=:log, legend=:topright, size=(900,600), 
    title="n=$n, \\gamma=$γ",
    xlabel="tol", 
    ylabel="function evaluations",
    xticks=(xticks, num2str.(xticks)), 
    ylim=[0, 3000], 
    yticks=0:1000:3000)

for (j, algo) in enumerate(algos)
    method, τ = algo
    plot!(plt, tols, mean(M[:,j,:], dims=2), label="$method", 
        ls=:auto, lc=:black)
end
plt

In [ ]:
savefig(plt, "figs/plot5.pdf")